In [1]:
from pycparser import parse_file, c_ast

global parents, variables
parents={}

In [ ]:
class Condition:
    def __init__(self, condition):
        (self.left_operand, self.operator, self.right_operand) = self.handle_condition(condition)
    
    def handle_condition(self, condition):
        left_operand = condition.left
        right_operand = condition.right
        operator = condition.op
        
        if operator == ">":
            return (right_operand, "<", left_operand)
        elif operator == ">=":
            return (right_operand, "<=", left_operand)
        else:
            return (left_operand, operator, right_operand)

class Context:  
    def __init__(self, previous_context = None, condition, next_node):
        if previous_context is not None:
            self.conditions = previous_context.conditions.append(condition, True)
            self.variables = previous_context.variables
            self.nexts = previous_context.nexts.append(next_node)
        else:
            self.conditions = [(condition, True)] # True si la condition a été vérifié, false sinon: permet de ne pas avoir de false positives
            self.variables = {}
            self.nexts = [next_node]
        self.next_contexts = []
    
    def handle_variable(self, variable, value):
        self.variables[variable] = value

In [2]:
def parse_c_file(file):
    ast = parse_file(file, use_cpp=True,
        cpp_path='gcc',
        cpp_args=['-E', r'-Iutils/fake_libc_include'])
    return ast

In [3]:
ast_graph = parse_c_file('./C files/if.c')

In [4]:
def computeParents(node, parent):
    global parents
    parents[node] = parent
    for (child, value) in node.children():
        computeParents(value, node) 

In [5]:
computeParents(ast_graph, None)

In [6]:
def findConditions(node):
    conditions = []
    if(type(node) == c_ast.While or type(node) == c_ast.If\
       or type(node) == c_ast.For):
        conditions.append(node.cond)
    for (child, value) in node.children():
        conditions += findConditions(value)
    return conditions

In [7]:
conditions = findConditions(ast_graph)

In [8]:
def getCoord(node):
    return node.coord

In [9]:
def findVariableDeclarations(node):
    declarations = []
    if(type(node) == c_ast.Decl):
        declarations.append(node)
    for (child, value) in node.children():
        declarations += findVariableDeclarations(value)
    return declarations

In [10]:
variableDeclarations = findVariableDeclarations(ast_graph)

In [17]:
def findAssignments(node):
    assignments = []
    if(type(node) == c_ast.Assignment):
        assignments.append(node)
    for (child, value) in node.children():
        assignments += findAssignments(value)
    return assignments

In [18]:
assignments = findAssignments(ast_graph)

In [ ]:
def visit_graph(graph, context = Context()):
    